<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import sys
import numpy as np
import os
import yaml
import matplotlib.pyplot as plt
import torchvision

In [2]:
%pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [3]:
def get_file_id_by_model(folder_name):
  file_id = {'resnet18_100-epochs_stl10': '14_nH2FkyKbt61cieQDiSbBVNP8-gtwgF',
             'resnet18_100-epochs_cifar10': '1lc2aoVtrAetGn0PnTkOyFzPCIucOJq7C',
             'resnet50_50-epochs_stl10': '1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu'}
  return file_id.get(folder_name, "Model not found.")

In [4]:
folder_name = 'resnet50_50-epochs_stl10'
file_id = get_file_id_by_model(folder_name)
print(folder_name, file_id)

resnet50_50-epochs_stl10 1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu


In [5]:
# download and extract model files
os.system('gdown https://drive.google.com/uc?id={}'.format(file_id))
os.system('unzip {}'.format(folder_name))
!ls

'ls' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [6]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [8]:
def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.STL10('./data', split='train', download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=3, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.STL10('./data', split='test', download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=3, drop_last=False, shuffle=shuffle)#changed num of workers
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.CIFAR10('./data', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=3, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('./data', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=3, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

In [9]:
# with open(os.path.join('./config.yml')) as file:
#   config = yaml.load(file)

class config_class:
  arch = "resnet18"
  dataset_name = "cifar10"
  def __init__(self):
    self.arch
    self.dataset_name

config = config_class()
config.arch = "resnet18"
config.dataset_name = "cifar10"

In [10]:
if config.arch == 'resnet18':
  model = torchvision.models.resnet18(pretrained=False, num_classes=10).to(device)
elif config.arch == 'resnet50':
  model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)

In [11]:
# checkpoint = torch.load('checkpoint_0040.pth.tar', map_location=device)
# state_dict = checkpoint['state_dict']

# for k in list(state_dict.keys()):

#   if k.startswith('backbone.'):
#     if k.startswith('backbone') and not k.startswith('backbone.fc'):
#       # remove prefix
#       state_dict[k[len("backbone."):]] = state_dict[k]
#   del state_dict[k]

In [12]:
# log = model.load_state_dict(state_dict, strict=False)
# assert log.missing_keys == ['fc.weight', 'fc.bias']

In [13]:
if config.dataset_name == 'cifar10':
  train_loader, test_loader = get_cifar10_data_loaders(download=True)
elif config.dataset_name == 'stl10':
  train_loader, test_loader = get_stl10_data_loaders(download=True)
print("Dataset:", config.dataset_name)

Files already downloaded and verified
Files already downloaded and verified
Dataset: cifar10


In [14]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.biase

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [16]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [17]:
epochs = 10
for epoch in range(epochs):
  print("Started epoch {}".format(epoch))
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(train_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)
    print("Batch {}".format(counter))
    print("x_batch:", x_batch.shape)
    print("y_batch:", y_batch.shape)
    

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top5_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy += top1[0]
    top5_accuracy += top5[0]
  
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")

Started epoch 0
Batch 0
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 1
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 2
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 3
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 4
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 5
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 6
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 7
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 8
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 9
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 10
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 11
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 12
x_batch: torch.Size([256, 3, 32, 32])
y_batch: torch.Size([256])
Batch 13
x_batch: torch.Size([25

In [18]:
#show the model layers
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
# from torchviz import make_dot
# y = model(img)
# make_dot(y, params=dict(list(auto_model.named_parameters()))).render("torchviz", format="png")

In [20]:
#Now we are going to freeze the weights of resnet 18
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

In [21]:
#verify weight are frozen

for name, param in joined_model.named_parameters():
    print(name, param.requires_grad)
print(joined_model.parameters)

NameError: name 'joined_model' is not defined